In [ ]:
def process_manual_entries(manual_entries, dois):
    """
    Process both manually entered records and DOIs for auto-retrieval.
    
    Parameters:
    manual_entries (list): List of manually created publication records
    dois (list): List of DOIs to automatically fetch metadata for
    
    Returns:
    list: Combined list of all manual entries (explicit and from DOIs)
    """
    all_entries = []
    
    # Add any predefined manual entries
    if manual_entries:
        all_entries.extend(manual_entries)
    
    # Process DOIs for automatic metadata retrieval
    if dois and len(dois) > 0:
        print(f"Attempting to automatically retrieve metadata for {len(dois)} DOIs...")
        for doi in dois:
            record = fetch_doi_metadata(doi)
            if record:
                all_entries.append(record)
    
    return all_entries


def fetch_doi_metadata(doi):
    """
    Fetch metadata for a DOI from Crossref or DataCite API.
    
    Parameters:
    doi (str): DOI to retrieve metadata for
    
    Returns:
    dict: Record in a format similar to PubMed records, or None if retrieval failed
    """
    import json
    import urllib.request
    import urllib.error
    
    # First try Crossref API
    url = f"https://api.crossref.org/works/{doi}"
    headers = {
        'User-Agent': 'PubMedSearchScript/1.0 (mailto:your.email@example.com)',
        'Accept': 'application/json'
    }
    
    try:
        # Make the request to the API
        req = urllib.request.Request(url, headers=headers)
        with urllib.request.urlopen(req, timeout=10) as response:
            data = json.loads(response.read().decode('utf-8'))
            
        # Extract the metadata from Crossref response
        if 'message' in data:
            metadata = data['message']
            
            # Process authors
            authors = []
            if 'author' in metadata:
                for author in metadata['author']:
                    if 'family' in author and 'given' in author:
                        # Format as "LastName InitialFirstName"
                        family = author['family']
                        given = author['given']
                        # Extract first initial
                        if given and len(given) > 0:
                            initial = given[0]
                            authors.append(f"{family} {initial}")
                        else:
                            authors.append(family)
            
            # Extract title
            title = ""
            if 'title' in metadata and metadata['title'] and len(metadata['title']) > 0:
                title = metadata['title'][0]
            
            # Extract journal
            journal = ""
            if 'container-title' in metadata and metadata['container-title'] and len(metadata['container-title']) > 0:
                journal = metadata['container-title'][0]
            
            # Extract publication date
            year = ""
            if 'published' in metadata and 'date-parts' in metadata['published']:
                date_parts = metadata['published']['date-parts']
                if date_parts and len(date_parts) > 0 and len(date_parts[0]) > 0:
                    year = str(date_parts[0][0])  # First part is year
            
            # Extract volume, issue, pages
            volume = metadata.get('volume', '')
            issue = metadata.get('issue', '')
            
            # Pages might be in different formats
            pages = ""
            if 'page' in metadata:
                pages = metadata['page']
            
            # Create a record similar to what PubMed would return
            record = {
                "PMID": f"DOI:{doi}",  # Use DOI as identifier
                "TI": title,
                "AU": authors,
                "JT": journal,
                "DP": year,
                "VI": volume,
                "IP": issue,
                "PG": pages,
                "LID": doi,
                "MANUAL": "False",  # Not manually entered
                "DOI": doi,
                "URL": f"https://doi.org/{doi}"
            }
            
            print(f"Successfully retrieved metadata for DOI: {doi}")
            return record
            
    except urllib.error.HTTPError as e:
        if e.code == 404:
            print(f"DOI not found: {doi}")
        else:
            print(f"HTTP Error when fetching DOI {doi}: {e.code} {e.reason}")
    except Exception as e:
        print(f"Error fetching metadata for DOI {doi}: {str(e)}")
    
    # If we got here, Crossref failed, try DataCite as a fallback
    try:
        url = f"https://api.datacite.org/works/{doi}"
        req = urllib.request.Request(url, headers=headers)
        with urllib.request.urlopen(req, timeout=10) as response:
            data = json.loads(response.read().decode('utf-8'))
            
        # Extract metadata from DataCite response
        if 'data' in data and 'attributes' in data['data']:
            metadata = data['data']['attributes']
            
            # Process authors
            authors = []
            if 'creators' in metadata:
                for creator in metadata['creators']:
                    if 'familyName' in creator and 'givenName' in creator:
                        family = creator['familyName']
                        given = creator['givenName']
                        # Extract first initial
                        if given and len(given) > 0:
                            initial = given[0]
                            authors.append(f"{family} {initial}")
                        else:
                            authors.append(family)
            
            # Extract title
            title = metadata.get('title', '')
            if isinstance(title, list) and title:
                title = title[0]
            
            # Extract journal/publisher
            journal = metadata.get('container-title', '')
            if not journal:
                journal = metadata.get('publisher', '')
            
            # Extract publication date
            year = ""
            if 'published' in metadata:
                year_match = re.search(r'(\d{4})', metadata['published'])
                if year_match:
                    year = year_match.group(1)
            
            # Create a record
            record = {
                "PMID": f"DOI:{doi}",
                "TI": title,
                "AU": authors,
                "JT": journal,
                "DP": year,
                "VI": metadata.get('volume', ''),
                "IP": metadata.get('issue', ''),
                "PG": metadata.get('pages', ''),
                "LID": doi,
                "MANUAL": "False",
                "DOI": doi,
                "URL": f"https://doi.org/{doi}"
            }
            
            print(f"Successfully retrieved metadata for DOI: {doi} (via DataCite)")
            return record
            
    except Exception as e:
        print(f"Error fetching DataCite metadata for DOI {doi}: {str(e)}")
    
    # If all methods failed, return None
    print(f"Failed to retrieve metadata for DOI: {doi}")
    return None


def create_manual_record(doi, title, authors, journal, year, volume="", issue="", pages="", url=None):
    """
    Create a manually defined record for publications not in PubMed.
    
    Parameters:
    doi (str): DOI of the publication
    title (str): Title of the publication
    authors (list): List of author names (format: "LastName FirstInitial")
    journal (str): Journal name or publication venue
    year (str): Publication year
    volume (str, optional): Volume number
    issue (str, optional): Issue number
    pages (str, optional): Page numbers
    url (str, optional): URL to the publication (if different from DOI URL)
    
    Returns:
    dict: Record in a format similar to PubMed records
    """
    # Create a record dictionary similar to what would be returned by PubMed
    record = {
        "PMID": f"DOI:{doi}",  # Use DOI as identifier since there's no PMID
        "TI": title,           # Title
        "AU": authors,         # Author list
        "JT": journal,         # Journal title
        "DP": year,            # Publication date (year)
        "VI": volume,          # Volume
        "IP": issue,           # Issue
        "PG": pages,           # Pages
        "LID": doi,            # DOI as a linkable ID
        "MANUAL": "True"       # Flag to identify manual entries
    }
    
    # Add URL if provided
    if url:
        record["URL"] = url
    else:
        record["URL"] = f"https://doi.org/{doi}"
        
    return record


import re
import sys
import ssl
import certifi
from collections import defaultdict
from datetime import datetime
from IPython.display import HTML, display, Markdown

try:
    from Bio import Entrez
    from Bio import Medline
    # Fix for SSL certificate verification issues
    import socket
    import urllib.request
    # Create a custom SSL context using certifi's certificates
    custom_context = ssl.create_default_context(cafile=certifi.where())
    
    # Monkey patch the default opener
    urllib.request.install_opener(
        urllib.request.build_opener(
            urllib.request.HTTPSHandler(context=custom_context)
        )
    )
except ImportError:
    print("This script requires Biopython and certifi. Please install them using:")
    print("!pip install biopython certifi")
    sys.exit(1)


def build_search_query(authors_list, affiliation, default_start_year, collaboration_groups=None):
    """
    Build a structured PubMed search query from a list of authors with individual date filters.
    
    Parameters:
    authors_list (list): List of author dictionaries with name, tag and optional start_year
    affiliation (str): Affiliation to search for (can be None to remove affiliation filter)
    default_start_year (str): Default start year for publication date filter
    collaboration_groups (list): List of research collaboration groups to include in search
    
    Returns:
    str: Formatted PubMed search query
    """
    # Process each author based on their tag type and start year
    author_queries = []
    
    for author in authors_list:
        name = author["name"]
        tag_type = author["tag"]
        
        # Get author-specific start year if available, otherwise use default
        start_year = author.get("start_year", default_start_year)
        
        # Create date filter for this author
        if start_year:
            date_filter = f"(\"{start_year}\"[Date - Publication] : \"3000\"[Date - Publication])"
        else:
            date_filter = ""  # No date filter for this author
        
        if tag_type == "author":
            # Standard author tag format
            author_query = f"({name}[Author])"
        elif tag_type == "custom":
            # Keep custom queries as-is
            author_query = f"({name})"
        else:
            # Default to author tag if not specified
            author_query = f"({name}[Author])"
            
        # Add date filter if specified
        if date_filter:
            author_query = f"({author_query} AND {date_filter})"
            
        author_queries.append(author_query)
    
    # Add collaboration group searches if provided
    if collaboration_groups:
        for group in collaboration_groups:
            author_queries.append(f"(\"{group}\"[Corporate Author])")
    
    # Join all author queries with OR
    authors_query = " OR ".join(author_queries)
    
    # Combine with affiliation if provided
    if affiliation:
        final_query = f"({affiliation}[Affiliation]) AND ({authors_query})"
    else:
        final_query = f"({authors_query})"  # No affiliation filter
    
    print(f"Generated query: {final_query}")
    
    return final_query
    
    return final_query


def search_pubmed(query, max_results=None, email="your.email@example.com", additional_pmids=None):
    """
    Search PubMed for the given query and return a list of PubMed IDs.
    
    Parameters:
    query (str): PubMed search query
    max_results (int, optional): Maximum number of results to retrieve (None for unlimited)
    email (str): Your email address (required by NCBI)
    additional_pmids (list): List of specific PMIDs to add to results
    
    Returns:
    list: List of PubMed records
    """
    Entrez.email = email
    all_records = []
    
    # First, directly fetch the specific PMIDs if provided
    if additional_pmids and len(additional_pmids) > 0:
        print(f"First retrieving {len(additional_pmids)} specific PMIDs...")
        # Start with these known publications
        specific_records = []
        for pmid in additional_pmids:
            try:
                handle = Entrez.efetch(db="pubmed", id=pmid, rettype="medline", retmode="text")
                specific_records.extend(list(Medline.parse(handle)))
                handle.close()
            except Exception as e:
                print(f"  Error retrieving PMID {pmid}: {str(e)}")
        print(f"Retrieved {len(specific_records)} specific publications")
        all_records.extend(specific_records)
    
    # Then, search PubMed to get the list of article IDs
    print(f"Searching PubMed for: {query}")
    try:
        # If max_results is None, use a very large number to get all results
        # PubMed API has a limit around 100,000 but we'll never reach that
        retmax = max_results if max_results is not None else 100000
        handle = Entrez.esearch(db="pubmed", term=query, retmax=retmax, sort="date")
        record = Entrez.read(handle)
        handle.close()
    except Exception as e:
        print(f"Error searching PubMed: {str(e)}")
        print("\nIf you're on macOS and having SSL certificate issues, try running this in a terminal:")
        print("python -m pip install --upgrade certifi")
        return all_records  # Return any records we already fetched directly
    
    id_list = record["IdList"]
    print(f"Found {len(id_list)} articles via query")
    
    # Don't fetch PMIDs we already have
    existing_pmids = [record.get("PMID", "") for record in all_records]
    id_list = [pmid for pmid in id_list if pmid not in existing_pmids]
    
    if not id_list:
        return all_records  # Return any records we already fetched directly
    
    # Then, fetch the full records for these IDs
    print(f"Retrieving details for {len(id_list)} additional articles...")
    try:
        # For large result sets, fetch in batches to avoid timeout issues
        batch_size = 500
        for i in range(0, len(id_list), batch_size):
            batch = id_list[i:i+batch_size]
            print(f"  Fetching batch {i//batch_size + 1} of {(len(id_list)-1)//batch_size + 1} ({len(batch)} articles)...")
            handle = Entrez.efetch(db="pubmed", id=batch, rettype="medline", retmode="text")
            batch_records = list(Medline.parse(handle))
            handle.close()
            all_records.extend(batch_records)
    except Exception as e:
        print(f"Error fetching article details: {str(e)}")
    
    print(f"Total publications found: {len(all_records)}")
    return all_records


def group_by_year(records):
    """Group records by publication year."""
    grouped = defaultdict(list)
    
    for record in records:
        # Extract year from the publication date
        if "DP" in record:
            # Publication date format can vary, so we extract just the year
            year_match = re.search(r'\b(19|20)\d{2}\b', record["DP"])
            if year_match:
                year = year_match.group(0)
                grouped[year].append(record)
            else:
                grouped["Unknown Year"].append(record)
        else:
            grouped["Unknown Year"].append(record)
    
    return grouped


def normalize_author_name(name):
    """
    Normalize author names to handle variations in capitalization and special characters.
    This helps with more consistent matching across different formats.
    """
    # Convert to lowercase for case-insensitive matching
    normalized = name.lower()
    # Replace hyphen with space to catch both hyphenated and non-hyphenated versions
    normalized = normalized.replace('-', ' ')
    return normalized


def is_team_member(author, authors_to_highlight):
    """
    Check if an author is a team member using normalized name matching.
    This handles different ways names might appear in PubMed.
    """
    # Normalize the author name for more consistent matching
    normalized_author = normalize_author_name(author)
    
    for highlight_name in authors_to_highlight:
        # Skip empty highlight names
        if not highlight_name:
            continue
            
        # Split the highlight name (which should be in LastName Initial format)
        highlight_parts = highlight_name.split() if isinstance(highlight_name, str) else []
        
        # Check for last name match (first part of the highlight name)
        if len(highlight_parts) > 0:
            last_name = normalize_author_name(highlight_parts[0])
            
            # Check if normalized last name is in the normalized author string
            if last_name in normalized_author:
                # If we have a last name match, check for initial if available
                if len(highlight_parts) > 1:
                    initial = highlight_parts[1][0].lower() if len(highlight_parts[1]) > 0 else ""
                    
                    # For initials, we look for them after the last name
                    if initial:
                        # Look for the initial after the last name mention
                        last_name_pos = normalized_author.find(last_name)
                        if last_name_pos >= 0:
                            remaining_text = normalized_author[last_name_pos + len(last_name):]
                            if initial.lower() in remaining_text.lower():
                                return True
                    else:
                        # If no initial provided, last name match is enough
                        return True
                else:
                    # If we only have a last name to match, that's enough
                    return True
    
    return False


def highlight_author(author, authors_to_highlight, format_type):
    """Highlight an author name if it appears in the list of authors to highlight."""
    if is_team_member(author, authors_to_highlight):
        if format_type == "text":
            return f"**{author}**"
        elif format_type == "html":
            return f'<span style="color: red; font-weight: bold;">{author}</span>'
        elif format_type == "markdown":
            return f"**{author}**"
    
    return author


def get_pubmed_link(pmid, format_type):
    """Generate a link to the PubMed article or DOI URL for manual entries."""
    # Check if this is a manual entry (DOI-based)
    if pmid.startswith("DOI:"):
        doi = pmid[4:]  # Remove the "DOI:" prefix
        url = f"https://doi.org/{doi}"
        link_text = "DOI"
    else:
        # Regular PubMed entry
        url = f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/"
        link_text = "PubMed"
    
    if format_type == "text":
        return url
    elif format_type == "html":
        return f'<a href="{url}" target="_blank" style="text-decoration: none;">{url}</a>'
    elif format_type == "markdown":
        # Using HTML inside markdown to remove the underline
        return f'<a href="{url}" style="text-decoration: none;">{link_text}</a>'
    
    return url


def format_record(record, authors_to_highlight, format_type):
    """Format a single record with highlighted authors."""
    # Get basic citation information
    title = record.get("TI", "No Title Available")
    journal = record.get("JT", "Unknown Journal")
    volume = record.get("VI", "")
    issue = record.get("IP", "")
    pages = record.get("PG", "")
    pmid = record.get("PMID", "")
    
    # Format journal info
    journal_info = journal
    if volume:
        journal_info += f", {volume}"
    if issue:
        journal_info += f"({issue})"
    if pages:
        journal_info += f":{pages}"
    
    # Format authors
    if "AU" in record:
        authors = record["AU"]
        formatted_authors = []
        
        for author in authors:
            formatted_authors.append(highlight_author(author, authors_to_highlight, format_type))
        
        if format_type == "text" or format_type == "markdown":
            authors_text = ", ".join(formatted_authors)
        elif format_type == "html":
            authors_text = ", ".join(formatted_authors)
    else:
        authors_text = "No authors listed"
    
    # Create link to PubMed
    pubmed_link = get_pubmed_link(pmid, format_type)
    
    # Format the complete record
    if format_type == "text":
        return f"* {title}\n  {authors_text}. {journal_info}. {pubmed_link}\n"
    
    elif format_type == "html":
        return f"""
        <li>
            <strong>{title}</strong><br>
            {authors_text}. <em>{journal_info}</em>. {pubmed_link}
        </li>
        <br>
        """
    
    elif format_type == "markdown":
        return f"* **{title}**  \n  {authors_text}. *{journal_info}*. {pubmed_link}\n"


def generate_output(grouped_records, authors_to_highlight, format_type, github_pages_header=None):
    """Generate formatted output."""
    if format_type == "html":
        output = "<html>\n<head>\n<title>PubMed Search Results</title>\n</head>\n<body>\n"
    elif format_type == "markdown" and github_pages_header:
        output = github_pages_header + "\n\n"
    else:
        output = ""
    
    # Sort years in descending order (most recent first)
    for year in sorted(grouped_records.keys(), reverse=True):
        records = grouped_records[year]
        
        if format_type == "html":
            output += f"<h2>{year}</h2>\n<ul>\n"
        elif format_type == "markdown":
            output += f"## {year}\n\n"
        else:
            output += f"{year}\n" + "-" * 40 + "\n"
        
        for record in records:
            formatted_record = format_record(record, authors_to_highlight, format_type)
            output += formatted_record + "\n"
        
        if format_type == "html":
            output += "</ul>\n"
        else:
            output += "\n"
    
    if format_type == "html":
        output += "</body>\n</html>"
    
    return output


def pubmed_search(query=None, email=None, authors_list=None, affiliation=None, 
                 default_start_year=None, highlight=None, max_results=None, 
                 output_format="markdown", output_file=None, github_pages_header=None,
                 collaboration_groups=None, additional_pmids=None, manual_entries=None,
                 auto_dois=None):
    """
    Main function for Jupyter notebooks to search PubMed and display results.
    
    Parameters:
    query (str, optional): PubMed search query (if provided, overrides authors_list and affiliation)
    email (str): Your email address (required by NCBI)
    authors_list (list): List of author dictionaries with name, tag, and optional start_year
    affiliation (str): Affiliation to search for
    default_start_year (str): Default start year for publication date filter
    highlight (list): List of author names to highlight
    max_results (int, optional): Maximum number of results to retrieve (None for unlimited)
    output_format (str): Output format ('text', 'html', or 'markdown')
    output_file (str): Optional file path to save the results
    github_pages_header (str): Optional GitHub Pages front matter header
    collaboration_groups (list): List of research collaboration groups to include in search
    additional_pmids (list): List of specific PMIDs to include regardless of search results
    manual_entries (list): List of manually defined records for publications not in PubMed
    auto_dois (list): List of DOIs to automatically fetch metadata for
    """
    if highlight is None:
        highlight = []
    
    # Build query if not provided
    if query is None and authors_list is not None:
        query = build_search_query(authors_list, affiliation, default_start_year, collaboration_groups)
    
    # Search PubMed
    records = search_pubmed(query, max_results, email, additional_pmids)
    
    # Process manual entries and auto DOIs
    all_manual_entries = process_manual_entries(manual_entries, auto_dois)
    
    # Add manual entries if there are any
    if all_manual_entries and len(all_manual_entries) > 0:
        print(f"Adding {len(all_manual_entries)} manual entries...")
        records.extend(all_manual_entries)
        print(f"Total publications including manual entries: {len(records)}")
    
    if not records:
        print("No results found for the query or an error occurred")
        return
    
    # Group records by year
    grouped_records = group_by_year(records)
    
    # Generate output
    output = generate_output(grouped_records, highlight or authors_list, output_format, github_pages_header)
    
    # Save to file if requested
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(output)
        print(f"Results saved to {output_file}")
    
    # Display results based on format
    if output_format == "html":
        display(HTML(output))
    elif output_format == "markdown":
        display(Markdown(output))
    else:
        print(output)
    
    return grouped_records  # Return data for further processing if needed


# CONFIGURATION SECTION
# ===========================================
# Set your parameters here in an easy-to-edit format

# CONFIGURATION SETTINGS
# ===========================================

# Your email for NCBI
YOUR_EMAIL = "nora.bruns@uk-essen.de"

# Affiliation to search - set to None to search without affiliation filter
# Change to None to find publications without the city in the affiliation
AFFILIATION = None  # Was "Essen"

# No default start year - each author can have their own
DEFAULT_START_YEAR = None

# Maximum results to retrieve (set to None for unlimited)
MAX_RESULTS = None  # No limit, retrieve all matching publications

# OUTPUT SETTINGS
# ===========================================
OUTPUT_FORMAT = "markdown"  # Options: "text", "html", "markdown"
OUTPUT_FILE = "index.md"  # Set to None to disable file output

# List of authors to search for - easy to add/edit
# Each author is a dictionary with:
#   - name: Author name in "LastName FirstName" format for precise searching
#   - highlight: Name format for highlighting (typically "LastName Initial")
#   - tag: "author" for standard [Author] tag, "custom" for custom query
#   - start_year: (Optional) Start year to filter publications for this author
AUTHORS_LIST = [
    # Dohna-Schwake with different variations to catch hyphenation and capitalization issues
    {"name": "Dohna-Schwake Christian", "highlight": "Dohna-Schwake C", "tag": "author", "start_year": "2019"},
    {"name": "Dohna Schwake Christian", "highlight": "Dohna-Schwake C", "tag": "author", "start_year": "2019"},
    {"name": "dohna-schwake c", "highlight": "Dohna-Schwake C", "tag": "custom", "start_year": "2019"},
    
    # Bruns variations
    {"name": "Bruns Nora", "highlight": "Bruns N", "tag": "author"},
    {"name": "bruns n essen", "highlight": "Bruns N", "tag": "custom"},  # Kept as requested
    
    # Other team members
    {"name": "Schwarz Simone", "highlight": "Schwarz S", "tag": "author", "start_year": "2022"},
    
    # Goretzki with variations to catch collaborative papers
    {"name": "Goretzki Sarah", "highlight": "Goretzki S", "tag": "author"},
    {"name": "Goretzki Sarah Christina", "highlight": "Goretzki SC", "tag": "author"},
    {"name": "Goretzki S", "highlight": "Goretzki S", "tag": "custom"},  # Without [Author] tag for better matching

    # Remaining team members
    {"name": "Greve Sandra", "highlight": "Greve S", "tag": "author"},
    {"name": "Brensing Pia", "highlight": "Brensing P", "tag": "author"},
    {"name": "Paul Luisa", "highlight": "Paul L", "tag": "author"},
    {"name": "Hojeij Rayan", "highlight": "Hojeij R", "tag": "author"}
]


# Specific PMIDs that should be included regardless of search results
# Add any known papers that aren't being found automatically
ADDITIONAL_PMIDS = [
    "36670127",  # Goretzki S paper example
    "31444505"   # Dohna-Schwake C paper example
]

# Manually define publications not in PubMed using DOIs
# Each entry should include required fields: doi, title, authors, journal, year
# Optional fields: volume, issue, pages, url (if different from DOI URL)
MANUAL_ENTRIES = [
    # Example of a manually defined publication (uncomment and modify as needed)
    # create_manual_record(
    #     doi="10.1000/example123",
    #     title="Example Publication Not in PubMed",
    #     authors=["Bruns N", "Schwarz S", "Dohna-Schwake C"],
    #     journal="Journal of Medical Research",
    #     year="2023",
    #     volume="15",
    #     issue="2", 
    #     pages="123-130"
    # ),
    # Add more manual entries as needed
]

# List of DOIs to automatically fetch metadata for
# Just add the DOI strings here, and the script will retrieve all publication details
AUTO_DOIS = [
     "10.1055/a-2479-2946",  # Sporttherapie B&G
     "10.1007/s00112-022-01482-z",  # Handlungsempfehlung
]

# Extract highlight names for the output
AUTHORS_TO_HIGHLIGHT = [author["highlight"] for author in AUTHORS_LIST]

# GitHub Pages YAML front matter header
GITHUB_HEADER = """---
type: page
title: Publications by our lab members
---"""

# OUTPUT SETTINGS
# ===========================================
OUTPUT_FORMAT = "markdown"  # Options: "text", "html", "markdown"
OUTPUT_FILE = "index.md"  # Set to None to disable file output

# RUN THE SEARCH
# ===========================================
# Build the search query and run the search
pubmed_search(
    email=YOUR_EMAIL,
    authors_list=AUTHORS_LIST,
    affiliation=AFFILIATION,
    default_start_year=DEFAULT_START_YEAR,
    highlight=AUTHORS_TO_HIGHLIGHT,
    max_results=MAX_RESULTS,
    output_format=OUTPUT_FORMAT,
    output_file=OUTPUT_FILE,
    github_pages_header=GITHUB_HEADER,
    collaboration_groups=COLLABORATION_GROUPS,
    additional_pmids=ADDITIONAL_PMIDS,
    manual_entries=MANUAL_ENTRIES,
    auto_dois=AUTO_DOIS
)

Generated query: (((Dohna-Schwake Christian[Author]) AND ("2019"[Date - Publication] : "3000"[Date - Publication])) OR ((Dohna Schwake Christian[Author]) AND ("2019"[Date - Publication] : "3000"[Date - Publication])) OR ((dohna-schwake c) AND ("2019"[Date - Publication] : "3000"[Date - Publication])) OR (Bruns Nora[Author]) OR (bruns n essen) OR ((Schwarz Simone[Author]) AND ("2022"[Date - Publication] : "3000"[Date - Publication])) OR (Goretzki Sarah[Author]) OR (Goretzki Sarah Christina[Author]) OR (Goretzki S) OR (Greve Sandra[Author]) OR (Brensing Pia[Author]) OR (Paul Luisa[Author]) OR (Hojeij Rayan[Author]) OR ("German Neonatal Network"[Corporate Author]) OR ("GNN"[Corporate Author]))
First retrieving 2 specific PMIDs...
Retrieved 2 specific publications
Searching PubMed for: (((Dohna-Schwake Christian[Author]) AND ("2019"[Date - Publication] : "3000"[Date - Publication])) OR ((Dohna Schwake Christian[Author]) AND ("2019"[Date - Publication] : "3000"[Date - Publication])) OR ((do

---
type: page
title: Publications by our lab members
---

## 2025

* **Congenital infection with Plasmodium malariae: a rare case of intrauterine transmission in Germany.**  
  **Goretzki S**, **Bruns N**, Daniels A, Schonecker A, Della Marina A, Gangfuss A, Schweiger B, Schonfeld A, Witzke O, Dedy J, Dziobaka J, Holtkamp C, Rath PM, Felderhoff-Muser U, **Dohna-Schwake C**, Verhasselt HL. *Malaria journal, 24(1):91*. <a href="https://pubmed.ncbi.nlm.nih.gov/40114164/" style="text-decoration: none;">PubMed</a>

* **The Phoenix Sepsis Score in Pediatric Oncology Patients With Sepsis at PICU Admission: Test of Performance in a European Multicenter Cohort, 2018-2020.**  
  Wosten-van Asperen RM, la Roi-Teeuw HM, Tissing WJE, Jordan I, **Dohna-Schwake C**, Bottari G, Pappachan J, Crazzolara R, Amigoni A, Mizia-Malarz A, Moscatelli A, Sanchez-Martin M, Willems J, Schlapbach LJ. *Pediatric critical care medicine : a journal of the Society of Critical Care Medicine and the World Federation of Pediatric Intensive and Critical Care Societies, 26(2):e177-e185*. <a href="https://pubmed.ncbi.nlm.nih.gov/39982155/" style="text-decoration: none;">PubMed</a>

* **Ultrasound Super-Resolution Imaging of Neonatal Cerebral Vascular Reorganization.**  
  **Schwarz S**, Denis L, Nedoschill E, Buehler A, Danko V, Hilger AC, Brevis Nunez F, Durr NR, Schlunz-Hendann M, Brassel F, Felderhoff-Muser U, Reutter H, Woelfle J, Jungert J, **Dohna-Schwake C**, **Bruns N**, Regensburger AP, Couture O, Mandelbaum H, Knieling F. *Advanced science (Weinheim, Baden-Wurttemberg, Germany), 12(12):e2415235*. <a href="https://pubmed.ncbi.nlm.nih.gov/39899647/" style="text-decoration: none;">PubMed</a>

* **Performance of ICD-10-based injury severity scores in pediatric trauma patients using the ICD-AIS map and survival rate ratios.**  
  **Hojeij R**, **Brensing P**, Nonnemacher M, Kowall B, Felderhoff-Muser U, Dudda M, **Dohna-Schwake C**, Stang A, **Bruns N**. *Journal of clinical epidemiology, 178:111634*. <a href="https://pubmed.ncbi.nlm.nih.gov/39647538/" style="text-decoration: none;">PubMed</a>

* **Severity of Pediatric Inflammatory Multisystem Syndrome Temporally Associated with SARS-CoV-2 Diminished During Successive Waves of the COVID-19 Pandemic: Data from a Nationwide German Survey.**  
  Lohrmann F, Doenhardt M, Diffloth N, Jakob A, Hospach A, Schneider DT, Trotter A, Brunner J, **Goretzki S**, Arens S, Rank M, Mauer R, Armann J, Berner R, Hufnagel M. *The Journal of pediatrics, 278:114419*. <a href="https://pubmed.ncbi.nlm.nih.gov/39603520/" style="text-decoration: none;">PubMed</a>

* **Management and outcome of cutaneous diphtheria in adolescent refugees in Germany, June 2022 - October 2023.**  
  Zink A, Hofer J, Schneider C, Kessler F, Klenze H, Klauwer D, Maleki K, Muller A, **Goretzki S**, Wang S, Kobbe R, Ramirez AV, Bode S, Janda A, Fressle R, Remppis J, Henneke P, Rieg S, Berger A, Sing A, Hufnagel M, Spielberger BD. *Infection, 53(1):329-337*. <a href="https://pubmed.ncbi.nlm.nih.gov/39190269/" style="text-decoration: none;">PubMed</a>

* **An [illustrative] update on pediatric emergency medicine ultrasound: part 2 - abdominal and urogenital applications.**  
  Dong Y, Hoffmann B, **Schwarz S**, Nourkami-Tutdibi N, Huang YL, Chen S, Cekuolis A, Augustiniene R, Snelling PJ, Schreiber-Dietrich D, Grevelding L, Dietrich CF. *Medical ultrasonography, 27(1):63-72*. <a href="https://pubmed.ncbi.nlm.nih.gov/39078993/" style="text-decoration: none;">PubMed</a>

* **Empfehlungen zur Planung und Durchführung einer Bewegungsintervention
                für Kinder und Jugendliche mit Long COVID**  
  Bergelt M, Eckert K, Gauß G, **Bruns N**, Benson S, Felderhoff-Müser U, **Dohna-Schwake C**, Brasseler M, **Goretzki S**. *B&amp;G Bewegungstherapie und Gesundheitssport, 41(01):30-37*. <a href="https://doi.org/10.1055/a-2479-2946" style="text-decoration: none;">DOI</a>


## 2024

* **Case report: Acute liver failure during deferasirox therapy and the potential role of pharmacogenetics.**  
  Garcia-Farina B, Rink L, Santarini V, Westkemper M, **Dohna-Schwake C**, Mohlendick B. *Frontiers in pharmacology, 15:1477755*. <a href="https://pubmed.ncbi.nlm.nih.gov/39508042/" style="text-decoration: none;">PubMed</a>

* **Ultrasound Localization Microscopy for the Assessment of Microvascular Circulation in Ischemic Perinatal Stroke.**  
  Regensburger AP, Wachter F, Denis L, Mandelbaum H, Schey F, Buehler A, Siebenlist G, **Schwarz S**, Schulz-Heise S, Rompel O, Trollmann R, Woelfle J, Jungert J, Couture O, Reutter H, Hanslik G, Knieling F. *Stroke, 55(12):e323-e325*. <a href="https://pubmed.ncbi.nlm.nih.gov/39474686/" style="text-decoration: none;">PubMed</a>

* **The effect of live-performed music therapy with physical contact in preterm infants on parental perceived stress and salivary cortisol levels.**  
  Dewan MV, Ader M, Kleinbeck T, Dathe AK, Schedlowski M, Engler H, Felderhoff-Mueser U, **Bruns N**, Kobus S. *Frontiers in psychology, 15:1441824*. <a href="https://pubmed.ncbi.nlm.nih.gov/39434912/" style="text-decoration: none;">PubMed</a>

* **How to perform Point of Care Ultrasound at resuscitation and when it is useful.**  
  Wastl D, Blaivas M, Horn R, Michels G, Seibel A, **Schwarz S**, Hoffmann BH, Hoffmann O, Von Ow D, Dietrich CF. *Medical ultrasonography*. <a href="https://pubmed.ncbi.nlm.nih.gov/39420822/" style="text-decoration: none;">PubMed</a>

* **An [illustrative] update on pediatric emergency ultrasound: part 3 - cerebral, musculoskeletal and other applications.**  
  **Schwarz S**, Dong Y, Snelling PJ, Hoffmann B, Nourkami-Tutdibi N, Huang YL, Chen S, Cekuolis A, Augustiniene R, Schreiber-Dietrich D, Grevelding L, Dietrich CF. *Medical ultrasonography*. <a href="https://pubmed.ncbi.nlm.nih.gov/39420818/" style="text-decoration: none;">PubMed</a>

* **Telemedicine-based antibiotic stewardship program in pediatrics: study protocol of a stepped wedge cluster randomized trial-the TeleKasper study.**  
  Diexer S, Ihling A, Gomes D, Moritz S, Simon A, **Dohna-Schwake C**, Mikolajczyk R, Huebner J, von Both U. *Trials, 25(1):678*. <a href="https://pubmed.ncbi.nlm.nih.gov/39402608/" style="text-decoration: none;">PubMed</a>

* **Feasibility, efficacy, and safety of animal-assisted activities with visiting dogs in inpatient pediatric oncology.**  
  Steff K, Grasemann M, Ostermann K, **Goretzki SC**, Rath PM, Reinhardt D, Schundeln MM. *World journal of pediatrics : WJP, 20(9):915-924*. <a href="https://pubmed.ncbi.nlm.nih.gov/39112809/" style="text-decoration: none;">PubMed</a>

* **Neonatal Sepsis Episodes and Retinopathy of Prematurity in Very Preterm Infants.**  
  Glaser K, Hartel C, Klingenberg C, Herting E, Fortmann MI, Speer CP, Stensvold HJ, Huncikova Z, Ronnestad AE, Nentwich MM, Stahl A, Dammann O, Gopel W. *JAMA network open, 7(7):e2423933*. <a href="https://pubmed.ncbi.nlm.nih.gov/39052290/" style="text-decoration: none;">PubMed</a>

* **Surgical Site Infections After Pediatric Liver Transplantation-Impact of a Change in Perioperative Prophylactic Antibiotic Protocol.**  
  **Bruns N**, Hauschild J, Lainka E, **Dohna-Schwake C**. *Pediatric transplantation, 28(5):e14830*. <a href="https://pubmed.ncbi.nlm.nih.gov/39030994/" style="text-decoration: none;">PubMed</a>

* **Short-term outcomes of asphyxiated neonates depending on requirement for transfer in the first 24 h of life.**  
  **Bruns N**, Feddahi N, **Hojeij R**, Rossi R, **Dohna-Schwake C**, Stein A, Kobus S, Stang A, Kowall B, Felderhoff-Muser U. *Resuscitation, 202:110309*. <a href="https://pubmed.ncbi.nlm.nih.gov/39002696/" style="text-decoration: none;">PubMed</a>

* **The effect of live music therapy on white matter microstructure in very preterm infants - A randomized controlled trial.**  
  Dewan MV, Jungilligens J, Kobus S, Diezel M, Dathe AK, Schweiger B, Huning B, Felderhoff-Muser U, **Bruns N**. *European journal of paediatric neurology : EJPN : official journal of the European Paediatric Neurology Society, 51:132-139*. <a href="https://pubmed.ncbi.nlm.nih.gov/38941879/" style="text-decoration: none;">PubMed</a>

* **Cerebral Doppler imaging in neonates: A guide for clinical application and diagnosis.**  
  Horsch S, **Schwarz S**, Arnaez J, Steggerda S, Arena R, Govaert P. *Developmental medicine and child neurology, 66(12):1570-1589*. <a href="https://pubmed.ncbi.nlm.nih.gov/38940604/" style="text-decoration: none;">PubMed</a>

* **Association of bilaterally suppressed EEG amplitudes and outcomes in critically ill children.**  
  **Paul L**, **Greve S**, Hegemann J, Gienger S, Loffelhardt VT, Della Marina A, Felderhoff-Muser U, **Dohna-Schwake C**, **Bruns N**. *Frontiers in neuroscience, 18:1411151*. <a href="https://pubmed.ncbi.nlm.nih.gov/38903601/" style="text-decoration: none;">PubMed</a>

* **Position statement on genuine physiotherapy research at German university hospitals.**  
  Klotz SGR, Bokel A, Friderichs-Nedohibchenko M, Stickdorn I, Vogel B, Doods B, Feldmann F, Ghiazza M, Giehl M, Hoberg A, Jansen L, Kohlhofer D, Leonhardt R, Meier SF, Muller C, Pannzek M, **Schwarz S**, Traut M, Urdahl M. *German medical science : GMS e-journal, 22:Doc06*. <a href="https://pubmed.ncbi.nlm.nih.gov/38883338/" style="text-decoration: none;">PubMed</a>

* **An [illustrative] update on pediatric emergency medicine ultrasound: part 1 - trauma and thoracic applications.**  
  Dong Y, **Schwarz S**, Hoffmann B, Nourkami-Tutdibi N, Huang YL, Chen S, Cekuolis A, Augustiniene R, Snelling PJ, Schreiber-Dietrich D, Grevelding L, Dietrich CF. *Medical ultrasonography, 26(4):393-404*. <a href="https://pubmed.ncbi.nlm.nih.gov/38805622/" style="text-decoration: none;">PubMed</a>

* **Effects of an early transfer from incubator to a warming crib in very low birthweight preterm infants.**  
  **Greve S**, **Bruns N**, Dathe AK, Schuendeln MM, Felderhoff-Mueser U, Stein A. *BMC pediatrics, 24(1):319*. <a href="https://pubmed.ncbi.nlm.nih.gov/38724933/" style="text-decoration: none;">PubMed</a>

* **COMFORTneo scale in preterm infants during live performed music therapy-Difference between close physical contact and hand touch contact.**  
  Kobus S, Kleinbeck T, Ader M, Dewan MV, Dathe AK, Feddahi N, Felderhoff-Mueser U, **Bruns N**. *Frontiers in neuroscience, 18:1359769*. <a href="https://pubmed.ncbi.nlm.nih.gov/38606306/" style="text-decoration: none;">PubMed</a>

* **Correlation of Comfort Score and Narcotrend Index during Procedural Sedation with Midazolam and Propofol in Children.**  
  **Bruns N**, Joist CA, Joist CM, Daniels A, Felderhoff-Muser U, **Dohna-Schwake C**, Tschiedel E. *Journal of clinical medicine, 13(5)*. <a href="https://pubmed.ncbi.nlm.nih.gov/38592307/" style="text-decoration: none;">PubMed</a>

* **Adaption of Follow-Up Visits for Preterm and High-Risk Children for Video Consultations.**  
  Dathe AK, **Greve S**, Teschler U, Heuser-Spura KM, Cordier L, Albayrak B, Felderhoff-Mueser U, Huening BM. *Telemedicine journal and e-health : the official journal of the American Telemedicine Association, 30(7):e2004-e2012*. <a href="https://pubmed.ncbi.nlm.nih.gov/38466975/" style="text-decoration: none;">PubMed</a>

* **Pediatric Community-Acquired Brain Abscesses: A Single-center Retrospective Study.**  
  Lu KH, **Bruns N**, Pentek C, Della Marina A, Gangfuss A, Kolbel H, Dogan B, Dziobaka J, Ahmadipour Y, Damman P, Felderhoff-Muser U, **Dohna-Schwake C**, **Goretzki SC**. *The Pediatric infectious disease journal, 43(6):e214-e217*. <a href="https://pubmed.ncbi.nlm.nih.gov/38451881/" style="text-decoration: none;">PubMed</a>

* **Current practice of intracranial pressure monitoring in children with severe traumatic brain injury-a nationwide prospective surveillance study in Germany.**  
  **Brensing P**, **Greve S**, **Hojeij R**, Dammann P, Felderhoff-Muser U, **Dohna-Schwake C**, **Bruns N**. *Frontiers in pediatrics, 12:1355771*. <a href="https://pubmed.ncbi.nlm.nih.gov/38405592/" style="text-decoration: none;">PubMed</a>

* **Outbreak of severe community-acquired bacterial infections among children in North Rhine-Westphalia (Germany), October to December 2022.**  
  **Goretzki SC**, van der Linden M, Itzek A, Huhne T, Adelmann RO, Ala Eldin F, Albarouni M, Becker JC, Berghauser MA, Boesing T, Boeswald M, Brasche M, Brevis Nunez F, Camara R, Deibert C, Dohle F, Dolgner J, Dziobaka J, Eifinger F, Elting N, Endmann M, Engelmann G, Frenzke H, Gappa M, Gharavi B, Goletz C, Hahn E, Heidenreich Y, Heimann K, Hensel KO, Hoffmann HG, Hoppenz M, Horneff G, Klassen H, Koerner-Rettberg C, Langler A, Lenz P, Lohmeier K, Muller A, Niemann F, Paulussen M, Pentek F, Perez R, Pingel M, Repges P, Rothoeft T, Rubo J, Schade H, Schmitz R, Schonhoff P, Schwade JN, Schwarz T, Seiffert P, Selzer G, Spille U, Thiel C, Thimm A, Urgatz B, van den Heuvel A, van Hop T, Giesen V, Wirth S, Wollbrink T, Wuller D, Felderhoff-Muser U, **Dohna-Schwake C**, Lam TT, Claus H, **Bruns N**. *Infection, 52(3):1099-1111*. <a href="https://pubmed.ncbi.nlm.nih.gov/38366304/" style="text-decoration: none;">PubMed</a>

* **Morphological chest CT changes in cystic fibrosis and massive hemoptysis.**  
  Dohna M, Kuhl H, Sutharsan S, **Bruns N**, Vo Chieu VD, Hellms S, Kornemann N, Montag MJ. *Radiologie (Heidelberg, Germany), 64(Suppl 1):76-85*. <a href="https://pubmed.ncbi.nlm.nih.gov/38326635/" style="text-decoration: none;">PubMed</a>

* **Clinical recommendations for the inpatient management of lower respiratory tract infections in children and adolescents with severe neurological impairment in Germany.**  
  Mauritz MD, von Both U, **Dohna-Schwake C**, Gille C, Hasan C, Huebner J, Hufnagel M, Knuf M, Liese JG, Renk H, Rudolph H, Schulze-Sturm U, Simon A, Stehling F, Tenenbaum T, Zernikow B. *European journal of pediatrics, 183(3):987-999*. <a href="https://pubmed.ncbi.nlm.nih.gov/38172444/" style="text-decoration: none;">PubMed</a>

* **[Increased rate of complications of pediatric acute otitis media and sinusitis in 2022/2023].**  
  Voss N, Sadok N, **Goretzki S**, **Dohna-Schwake C**, Meyer MF, Mattheis S, Lang S, Stahr K. *HNO, 72(2):83-89*. <a href="https://pubmed.ncbi.nlm.nih.gov/38108853/" style="text-decoration: none;">PubMed</a>

* **Targeted PCR of Mucorales in pediatric bronchoalveolar lavage samples indicates low prevalence of airway colonization and sample contamination.**  
  Kavvalou A, Stehling F, **Goretzki SC**, **Dohna-Schwake C**, Rath PM, Steindor M. *Infectious diseases (London, England), 56(3):255-258*. <a href="https://pubmed.ncbi.nlm.nih.gov/38015560/" style="text-decoration: none;">PubMed</a>

* **Aortic Steal Correlates with Acute Organ Dysfunction and Short-Term Outcomes in Neonates with Vein of Galen Malformation.**  
  **Schwarz S**, Brevis Nunez F, Durr NR, Schlunz-Hendann M, Brassel F, Felderhoff-Muser U, **Dohna-Schwake C**, **Bruns N**. *Neonatology, 121(1):106-115*. <a href="https://pubmed.ncbi.nlm.nih.gov/37906988/" style="text-decoration: none;">PubMed</a>


## 2023

* **A comparison of pediatric inflammatory multisystem syndrome temporarily-associated with SARS-CoV-2 and Kawasaki disease.**  
  Hufnagel M, Armann J, Jakob A, Doenhardt M, Diffloth N, Hospach A, Schneider DT, Trotter A, Roessler M, Schmitt J, Berner R. *Scientific reports, 13(1):1173*. <a href="https://pubmed.ncbi.nlm.nih.gov/36670127/" style="text-decoration: none;">PubMed</a>

* **Pediatric intensive care unit admissions network-rationale, framework and method of operation of a nationwide collaborative pediatric intensive care research network in Germany.**  
  **Bruns N**, **Dohna-Schwake C**, Olivieri M, Urschitz MS, Blomenkamp S, Frosch C, Lieftuchter V, Tomidis Chatzimanouil MK, Hoffmann F, Brenner S. *Frontiers in pediatrics, 11:1254935*. <a href="https://pubmed.ncbi.nlm.nih.gov/38269291/" style="text-decoration: none;">PubMed</a>

* **Increased SARS-CoV-2 reactive low avidity T cells producing inflammatory cytokines in pediatric post-acute COVID-19 sequelae (PASC).**  
  Paniskaki K, **Goretzki S**, Anft M, Konik MJ, Meister TL, Pfaender S, Lechtenberg K, Vogl M, Dogan B, Dolff S, Westhoff TH, Rohn H, Felderhoff-Mueser U, Stervbo U, Witzke O, **Dohna-Schwake C**, Babel N. *Pediatric allergy and immunology : official publication of the European Society of Pediatric Allergy and Immunology, 34(12):e14060*. <a href="https://pubmed.ncbi.nlm.nih.gov/38146118/" style="text-decoration: none;">PubMed</a>

* **Review on Pediatric Malignant Focal Liver Lesions with Imaging Evaluation: Part II.**  
  Dong Y, Cekuolis A, Schreiber-Dietrich D, Augustiniene R, **Schwarz S**, Moller K, Nourkami-Tutdibi N, Chen S, Cao JY, Huang YL, Wang Y, Taut H, Grevelding L, Dietrich CF. *Diagnostics (Basel, Switzerland), 13(24)*. <a href="https://pubmed.ncbi.nlm.nih.gov/38132242/" style="text-decoration: none;">PubMed</a>

* **Determinants of human papillomavirus vaccine hesitancy among Lebanese parents.**  
  Zakhour R, Tamim H, Faytrouni F, Makki M, **Hojeij R**, Charafeddine L. *PloS one, 18(12):e0295644*. <a href="https://pubmed.ncbi.nlm.nih.gov/38091310/" style="text-decoration: none;">PubMed</a>

* **Review on Pediatric Malignant Focal Liver Lesions with Imaging Evaluation: Part I.**  
  Dong Y, Cekuolis A, Schreiber-Dietrich D, Augustiniene R, **Schwarz S**, Moller K, Nourkami-Tutdibi N, Chen S, Cao JY, Huang YL, Wang Y, Taut H, Grevelding L, Dietrich CF. *Diagnostics (Basel, Switzerland), 13(23)*. <a href="https://pubmed.ncbi.nlm.nih.gov/38066809/" style="text-decoration: none;">PubMed</a>

* **Fading SARS-CoV-2 humoral VOC cross-reactivity and sustained cellular immunity in convalescent children and adolescents.**  
  Paniskaki K, **Goretzki S**, Anft M, Konik MJ, Lechtenberg K, Vogl M, Meister TL, Pfaender S, Zettler M, Jager J, Dolff S, Westhoff TH, Rohn H, Felderhoff-Mueser U, Stervbo U, Witzke O, **Dohna-Schwake C**, Babel N. *BMC infectious diseases, 23(1):818*. <a href="https://pubmed.ncbi.nlm.nih.gov/37993788/" style="text-decoration: none;">PubMed</a>

* **Music therapy modulates mothers' perception of their preterm infants.**  
  Kobus S, Diezel M, Dewan MV, Huening B, Dathe AK, Marschik PB, Felderhoff-Mueser U, **Bruns N**. *Frontiers in psychology, 14:1231741*. <a href="https://pubmed.ncbi.nlm.nih.gov/37928582/" style="text-decoration: none;">PubMed</a>

* **Distinct clinical phenotypes in paediatric cancer patients with sepsis are associated with different outcomes-an international multicentre retrospective study.**  
  Wosten-van Asperen RM, la Roi-Teeuw HM, van Amstel RB, Bos LD, Tissing WJ, Jordan I, **Dohna-Schwake C**, Bottari G, Pappachan J, Crazzolara R, Comoretto RI, Mizia-Malarz A, Moscatelli A, Sanchez-Martin M, Willems J, Rogerson CM, Bennett TD, Luo Y, Atreya MR, Faustino EVS, Geva A, Weiss SL, Schlapbach LJ, Sanchez-Pinto LN. *EClinicalMedicine, 65:102252*. <a href="https://pubmed.ncbi.nlm.nih.gov/37842550/" style="text-decoration: none;">PubMed</a>

* **[Time to Antibiotics (TTA) - Reassessment from the German Working Group for Fever and Neutropenia in Children and Adolescents (DGPI/GPOH)].**  
  Simon A, Lehrnbecher T, Baltaci Y, **Dohna-Schwake C**, Groll A, Laws HJ, Potratz J, Hufnagel M, Bochennek K. *Klinische Padiatrie, 235(6):331-341*. <a href="https://pubmed.ncbi.nlm.nih.gov/37751768/" style="text-decoration: none;">PubMed</a>

* **Erythrocyte transfusions are associated with retinopathy of prematurity in extremely low gestational age newborns.**  
  Glaser K, Hartel C, Dammann O, Herting E, Andres O, Speer CP, Gopel W, Stahl A. *Acta paediatrica (Oslo, Norway : 1992), 112(12):2507-2515*. <a href="https://pubmed.ncbi.nlm.nih.gov/37667535/" style="text-decoration: none;">PubMed</a>

* **Outcome after conservative or surgical treatment for new-onset epilepsy in children with cerebral cavernous malformation.**  
  Santos AN, Rauschenbach L, Riess C, Georgiades I, Ficilar B, Gallardo EG, Quesada CM, Li Y, Tippelt S, **Dohna-Schwake C**, Schmidt B, Jabbarli R, Siegel AM, Benet A, Wrede KH, Sure U, Dammann P. *Seizure, 111:23-29*. <a href="https://pubmed.ncbi.nlm.nih.gov/37494759/" style="text-decoration: none;">PubMed</a>

* **Early Prediction of Mortality after Birth Asphyxia with the nSOFA.**  
  Dathe AK, Stein A, **Bruns N**, Craciun ED, Tuda L, Bialas J, Brasseler M, Felderhoff-Mueser U, Huening BM. *Journal of clinical medicine, 12(13)*. <a href="https://pubmed.ncbi.nlm.nih.gov/37445355/" style="text-decoration: none;">PubMed</a>

* **Development of an Ultrasound Scoring System to Describe Brain Maturation in Preterm Infants.**  
  Stein A, Sody E, **Bruns N**, Felderhoff-Muser U. *AJNR. American journal of neuroradiology, 44(7):846-852*. <a href="https://pubmed.ncbi.nlm.nih.gov/37321856/" style="text-decoration: none;">PubMed</a>

* **Doppler Ultrasound Flow Reversal in the Superior Sagittal Sinus to Detect Cerebral Venous Congestion in Vein of Galen Malformation.**  
  **Schwarz S**, Brevis Nunez F, Durr NR, Brassel F, Schlunz-Hendann M, Feldkamp A, Rosenbaum T, Felderhoff-Muser U, Schulz K, **Dohna-Schwake C**, **Bruns N**. *AJNR. American journal of neuroradiology, 44(6):707-715*. <a href="https://pubmed.ncbi.nlm.nih.gov/37230540/" style="text-decoration: none;">PubMed</a>

* **Giant Cavernous Malformation Mimicking an Infiltrative Intracranial Neoplasm in Children-Case Report and Systematic Review of the Literature.**  
  Gonzalez-Gallardo E, Rauschenbach L, Santos AN, Riess C, Li Y, Tippelt S, Della Marina A, **Dohna-Schwake C**, Sure U, Dammann P. *World neurosurgery, 174:30-41*. <a href="https://pubmed.ncbi.nlm.nih.gov/36889633/" style="text-decoration: none;">PubMed</a>

* **High Prevalence of Alternative Diagnoses in Children and Adolescents with Suspected Long COVID-A Single Center Cohort Study.**  
  **Goretzki SC**, Brasseler M, Dogan B, Huhne T, Bernard D, Schonecker A, Steindor M, Gangfuss A, Della Marina A, Felderhoff-Muser U, **Dohna-Schwake C**, **Bruns N**. *Viruses, 15(2)*. <a href="https://pubmed.ncbi.nlm.nih.gov/36851793/" style="text-decoration: none;">PubMed</a>

* **A European International Multicentre Survey on the Current Practice of Perioperative Antibiotic Prophylaxis for Paediatric Liver Transplantations.**  
  Hauschild J, **Bruns N**, Lainka E, **Dohna-Schwake C**. *Antibiotics (Basel, Switzerland), 12(2)*. <a href="https://pubmed.ncbi.nlm.nih.gov/36830202/" style="text-decoration: none;">PubMed</a>

* **Functional outcome after initial and multiple intracerebral hemorrhages in children with cerebral cavernous malformations.**  
  Santos AN, Rauschenbach L, Gull HH, Dinger TF, Chihi M, Li Y, Tippelt S, **Dohna-Schwake C**, Schmidt B, Jabbarli R, Wrede KH, Sure U, Dammann P. *European journal of neurology, 30(5):1364-1370*. <a href="https://pubmed.ncbi.nlm.nih.gov/36789485/" style="text-decoration: none;">PubMed</a>

* **IgD shapes the pre-immune naive B cell compartment in humans.**  
  Dirks J, Andres O, **Paul L**, Manukjan G, Schulze H, Morbach H. *Frontiers in immunology, 14:1096019*. <a href="https://pubmed.ncbi.nlm.nih.gov/36776874/" style="text-decoration: none;">PubMed</a>

* **Functional outcome after pediatric cerebral cavernous malformation surgery.**  
  Rauschenbach L, Santos AN, Dinger TF, Darkwah Oppong M, Li Y, Tippelt S, **Dohna-Schwake C**, Schmidt B, Jabbarli R, Wrede KH, Sure U, Dammann P. *Scientific reports, 13(1):2286*. <a href="https://pubmed.ncbi.nlm.nih.gov/36759693/" style="text-decoration: none;">PubMed</a>

* **Pulmonary Sonography - Neonatal Diagnosis Part 2.**  
  **Schwarz S**. *Ultraschall in der Medizin (Stuttgart, Germany : 1980), 44(3):240-268*. <a href="https://pubmed.ncbi.nlm.nih.gov/36669528/" style="text-decoration: none;">PubMed</a>

* **[Pediatric neurocritical care].**  
  **Bruns N**, Schara-Schmidt U, **Dohna-Schwake C**. *Der Nervenarzt, 94(2):75-83*. <a href="https://pubmed.ncbi.nlm.nih.gov/36645451/" style="text-decoration: none;">PubMed</a>

* **Re: "Watch Out for Drug-Induced Coma and Burst Suppression Pattern in Infants and Children Mimicking Severe Neurological Disease" by Falsaperla et al.**  
  **Bruns N**. *Therapeutic hypothermia and temperature management, 13(1):42-43*. <a href="https://pubmed.ncbi.nlm.nih.gov/36576494/" style="text-decoration: none;">PubMed</a>

* **Feasibility of Video Consultation for Preterm Neurodevelopmental Follow-up Care During the COVID-19 Pandemic: Cohort Study.**  
  Albayrak B, Cordier LJ, **Greve S**, Teschler U, Dathe AK, Felderhoff-Muser U, Huning BM. *JMIR pediatrics and parenting, 6:e40940*. <a href="https://pubmed.ncbi.nlm.nih.gov/36409307/" style="text-decoration: none;">PubMed</a>

* **Pulmonary Sonography - Neonatal Diagnosis Part 1.**  
  **Schwarz S**. *Ultraschall in der Medizin (Stuttgart, Germany : 1980), 44(1):14-35*. <a href="https://pubmed.ncbi.nlm.nih.gov/36075237/" style="text-decoration: none;">PubMed</a>

* **Dissection of the Brachiocephalic Trunk with Subsequent Occlusion of Both Common Carotid Arteries in a Newborn with an Aortic Arch Variant - From Cerebral Doppler Ultrasound to Diagnosis.**  
  **Schwarz S**, Tarusinov G, Brevis F. *Ultraschall in der Medizin (Stuttgart, Germany : 1980), 44(3):e164-e167*. <a href="https://pubmed.ncbi.nlm.nih.gov/35211943/" style="text-decoration: none;">PubMed</a>


## 2022

* **Music Therapy in Preterm Infants Reduces Maternal Distress.**  
  Kobus S, Diezel M, Dewan MV, Huening B, Dathe AK, Marschik PB, Felderhoff-Mueser U, **Bruns N**. *International journal of environmental research and public health, 20(1)*. <a href="https://pubmed.ncbi.nlm.nih.gov/36613052/" style="text-decoration: none;">PubMed</a>

* **Organizational characteristics of European pediatric onco-critical care: An international cross-sectional survey.**  
  Nielsen JSA, Agbeko R, Bate J, Jordan I, **Dohna-Schwake C**, Potratz J, Moscatelli A, Bottari G, Pappachan J, Witt V, Crazzolara R, Amigoni A, Mizia-Malarz A, Sanchez Martin M, Willems J, van den Heuvel-Eibrink MM, Schlapbach LJ, Wosten-van Asperen RM. *Frontiers in pediatrics, 10:1024273*. <a href="https://pubmed.ncbi.nlm.nih.gov/36533247/" style="text-decoration: none;">PubMed</a>

* **Development of restrictive eating disorders in children and adolescents with long-COVID-associated smell and taste dysfunction.**  
  Brasseler M, Schonecker A, Steindor M, Della Marina A, **Bruns N**, Dogan B, Felderhoff-Muser U, Hebebrand J, **Dohna-Schwake C**, **Goretzki SC**. *Frontiers in pediatrics, 10:1022669*. <a href="https://pubmed.ncbi.nlm.nih.gov/36518782/" style="text-decoration: none;">PubMed</a>

* **Next Generation Sequencing of Free Microbial DNA for Rapid Identification of Pathogens in Critically Ill Children with Systemic Inflammatory Response Syndrome (SIRS).**  
  **Goretzki SC**, Schafer M, Dogan B, **Bruns N**, Tschiedel E, Rath PM, Voigt S, Grumaz S, Horvatek P, Schonberger S, Stehling F, Brenner T, **Dohna-Schwake C**. *Frontiers in bioscience (Landmark edition), 27(11):302*. <a href="https://pubmed.ncbi.nlm.nih.gov/36472101/" style="text-decoration: none;">PubMed</a>

* **Bronchial artery diameter in massive hemoptysis in cystic fibrosis.**  
  Dohna M, Kuhl H, Sutharsan S, **Dohna-Schwake C**, Vo Chieu VD, Hellms S, Kornemann N, Renz DM, Montag MJ. *BMC pulmonary medicine, 22(1):424*. <a href="https://pubmed.ncbi.nlm.nih.gov/36397043/" style="text-decoration: none;">PubMed</a>

* **Recurrent Late-Onset Sepsis in Extremely Low Birth Weight Infants Is Associated with Motor Deficits in Early School Age.**  
  Humberg A, Fortmann MI, Spiegler J, Rausch TK, Siller B, Silwedel C, Marissen J, Herting E, Gopel W, Hartel C. *Neonatology, 119(6):695-702*. <a href="https://pubmed.ncbi.nlm.nih.gov/36327925/" style="text-decoration: none;">PubMed</a>

* **Administrative data in pediatric critical care research-Potential, challenges, and future directions.**  
  **Bruns N**, Sorg AL, Felderhoff-Muser U, **Dohna-Schwake C**, Stang A. *Frontiers in pediatrics, 10:1014094*. <a href="https://pubmed.ncbi.nlm.nih.gov/36245724/" style="text-decoration: none;">PubMed</a>

* **The impact of age and electrode position on amplitude-integrated EEGs in children from 1 month to 17 years of age.**  
  **Greve S**, Loffelhardt VT, Della Marina A, Felderhoff-Muser U, **Dohna-Schwake C**, **Bruns N**. *Frontiers in neurology, 13:952193*. <a href="https://pubmed.ncbi.nlm.nih.gov/36090865/" style="text-decoration: none;">PubMed</a>

* **Sleep problems in infancy and early school age in very preterm infants.**  
  Neitmann J, Hanke K, Humberg A, Siller B, Spiegler J, Juhnke K, Gilmore J, Odendahl R, Herting E, Gopel W, Hartel C, Fortmann I. *Early human development, 173:105656*. <a href="https://pubmed.ncbi.nlm.nih.gov/35987047/" style="text-decoration: none;">PubMed</a>

* **Impact of Physical Contact on Preterm Infants' Vital Sign Response to Live Music Therapy.**  
  Kobus S, Diezel M, Dewan MV, Huening B, Dathe AK, Felderhoff-Mueser U, **Bruns N**. *International journal of environmental research and public health, 19(15)*. <a href="https://pubmed.ncbi.nlm.nih.gov/35954880/" style="text-decoration: none;">PubMed</a>

* **Association of Administration of Surfactant Using Less Invasive Methods With Outcomes in Extremely Preterm Infants Less Than 27 Weeks of Gestation.**  
  Hartel C, Herting E, Humberg A, Hanke K, Mehler K, Keller T, Mauer I, Frieauff E, Meyer S, Thome UH, Wieg C, Schmidtke S, Kribs A, Gopel W. *JAMA network open, 5(8):e2225810*. <a href="https://pubmed.ncbi.nlm.nih.gov/35943742/" style="text-decoration: none;">PubMed</a>

* **Prevalence of osteopathologies in a single center cohort of survivors of childhood primary brain tumor.**  
  Schundeln MM, Fritzemeier S, **Goretzki SC**, Hauffa PK, Munteanu M, Kiewert C, Hauffa BP, Fleischhack G, Tippelt S, Grasemann C. *Frontiers in pediatrics, 10:913343*. <a href="https://pubmed.ncbi.nlm.nih.gov/35923779/" style="text-decoration: none;">PubMed</a>

* **Pediatric ICU Admissions After Adolescent Suicide Attempts During the Pandemic.**  
  **Bruns N**, Willemsen L, Stang A, Kowall B, Holtkamp K, Kamp O, Dudda M, Hey F, Hoffmann F, Blankenburg J, Eifinger F, Fuchs H, Haase R, Baier J, Andree C, Heldmann M, Maldera V, Potratz J, Kurz D, Mand N, Doerfel C, Rothoeft T, Schultz M, Ohlert M, Silkenbaumer K, Boesing T, Indraswari F, Niemann F, Jahn P, Merker M, Braun N, Nunez FB, Engler M, Heimann K, Brasche M, Wolf G, Freymann H, Dercks M, Hoppenz M, Felderhoff-Muser U, **Dohna-Schwake C**. *Pediatrics, 150(2)*. <a href="https://pubmed.ncbi.nlm.nih.gov/35534988/" style="text-decoration: none;">PubMed</a>

* **Impact of the First COVID Lockdown on Accident- and Injury-Related Pediatric Intensive Care Admissions in Germany-A Multicenter Study.**  
  **Bruns N**, Willemsen LY, Holtkamp K, Kamp O, Dudda M, Kowall B, Stang A, Hey F, Blankenburg J, Sabir H, Eifinger F, Fuchs H, Haase R, Andree C, Heldmann M, Potratz J, Kurz D, Schumann A, Muller-Knapp M, Mand N, Doerfel C, Dahlem P, Rothoeft T, Ohlert M, Silkenbaumer K, Dohle F, Indraswari F, Niemann F, Jahn P, Merker M, Braun N, Brevis Nunez F, Engler M, Heimann K, Wolf GK, Wulf D, Hankel S, Freymann H, Allgaier N, Knirsch F, Dercks M, Reinhard J, Hoppenz M, Felderhoff-Muser U, **Dohna-Schwake C**. *Children (Basel, Switzerland), 9(3)*. <a href="https://pubmed.ncbi.nlm.nih.gov/35327736/" style="text-decoration: none;">PubMed</a>

* **Role of extracorporeal membrane oxygenation in pediatric cancer patients: a systematic review and meta-analysis of observational studies.**  
  Slooff V, Hoogendoorn R, Nielsen JSA, Pappachan J, Amigoni A, Caramelli F, Aziz O, Wildschut E, Verbruggen S, Crazzolara R, **Dohna-Schwake C**, Potratz J, Willems J, Llevadias J, Moscatelli A, Montaguti A, Bottari G, Di Nardo M, Schlapbach L, Wosten-van Asperen R. *Annals of intensive care, 12(1):8*. <a href="https://pubmed.ncbi.nlm.nih.gov/35092500/" style="text-decoration: none;">PubMed</a>

* **Breastfeeding and Readmission for Hyperbilirubinemia in Late Preterm and Term Infants in Beirut, Lebanon.**  
  Hanin EA, Rayan H, Hani T, Taleb J, Dany AH, Lama C. *Indian pediatrics, 59(3):218-221*. <a href="https://pubmed.ncbi.nlm.nih.gov/34969942/" style="text-decoration: none;">PubMed</a>

* **Functional Short-Term Outcomes and Mortality in Children with Severe Traumatic Brain Injury: Comparing Decompressive Craniectomy and Medical Management.**  
  **Bruns N**, Kamp O, Lange K, Lefering R, Felderhoff-Muser U, Dudda M, **Dohna-Schwake C**. *Journal of neurotrauma, 39(13-14):944-953*. <a href="https://pubmed.ncbi.nlm.nih.gov/34877889/" style="text-decoration: none;">PubMed</a>

* **Antibiotics in critically ill children-a narrative review on different aspects of a rational approach.**  
  **Bruns N**, **Dohna-Schwake C**. *Pediatric research, 91(2):440-446*. <a href="https://pubmed.ncbi.nlm.nih.gov/34873285/" style="text-decoration: none;">PubMed</a>

* **Natural Course of Cerebral Cavernous Malformations in Children: A Five-Year Follow-Up Study.**  
  Santos AN, Rauschenbach L, Saban D, Chen B, Herten A, Dinger TF, Li Y, Tippelt S, Della Marina A, **Dohna-Schwake C**, Schmidt B, Jabbarli R, Wrede KH, Sure U, Dammann P. *Stroke, 53(3):817-824*. <a href="https://pubmed.ncbi.nlm.nih.gov/34706565/" style="text-decoration: none;">PubMed</a>

* **Association of ApoE Genotypes and Recovery From Intracerebral Hemorrhage in Very Low Birth Weight Infants.**  
  Humberg A, Dzietko M, Schulz S, Felderhoff-Muser U, Huning B, Stein A, Fortmann MI, Marissen J, Rausch TK, Herting E, Hartel C, Gopel W. *Stroke, 53(2):514-522*. <a href="https://pubmed.ncbi.nlm.nih.gov/34619984/" style="text-decoration: none;">PubMed</a>

* **Latency duration of preterm premature rupture of membranes and neonatal outcome: a retrospective single-center experience.**  
  Muller H, Stahling AC, **Bruns N**, Weiss C, Ai M, Koninger A, Felderhoff-Muser U. *European journal of pediatrics, 181(2):801-811*. <a href="https://pubmed.ncbi.nlm.nih.gov/34605998/" style="text-decoration: none;">PubMed</a>

* **Handlungsempfehlung nach der S2k-Leitlinie Das Schädel-Hirn-Trauma im Kindes- und Jugendalter**  
  **Bruns N**, **Dohna-Schwake C**. *Monatsschrift Kinderheilkunde, 171(6):531-533*. <a href="https://doi.org/10.1007/s00112-022-01482-z" style="text-decoration: none;">DOI</a>


## 2021

* **Characterization of aEEG During Sleep and Wakefulness in Healthy Children.**  
  Loffelhardt VT, Della Marina A, **Greve S**, Muller H, Felderhoff-Muser U, **Dohna-Schwake C**, **Bruns N**. *Frontiers in pediatrics, 9:773188*. <a href="https://pubmed.ncbi.nlm.nih.gov/35127587/" style="text-decoration: none;">PubMed</a>

* **Parents' Perception of Family-Centered Music Therapy with Stable Preterm Infants.**  
  Kobus S, Diezel M, Huening B, Dewan MV, Felderhoff-Mueser U, **Bruns N**. *International journal of environmental research and public health, 18(23)*. <a href="https://pubmed.ncbi.nlm.nih.gov/34886539/" style="text-decoration: none;">PubMed</a>

* **Hospitalization and Morbidity Rates After Pediatric Traumatic Brain Injury: A Nation-Wide Population-Based Analysis.**  
  **Bruns N**, Trocchi P, Felderhoff-Muser U, **Dohna-Schwake C**, Stang A. *Frontiers in pediatrics, 9:747743*. <a href="https://pubmed.ncbi.nlm.nih.gov/34660495/" style="text-decoration: none;">PubMed</a>

* **Music Therapy Is Effective during Sleep in Preterm Infants.**  
  Kobus S, Diezel M, Dewan MV, Huening B, Dathe AK, Felderhoff-Mueser U, **Bruns N**. *International journal of environmental research and public health, 18(16)*. <a href="https://pubmed.ncbi.nlm.nih.gov/34443994/" style="text-decoration: none;">PubMed</a>

* **Coil embolisation for massive haemoptysis in cystic fibrosis.**  
  Dohna M, Renz DM, Stehling F, **Dohna-Schwake C**, Sutharsan S, Neurohr C, Wirtz H, Eickmeier O, Grosse-Onnebrink J, Sauerbrey A, Soditt V, Poplawska K, Wacker F, Montag MJ. *BMJ open respiratory research, 8(1)*. <a href="https://pubmed.ncbi.nlm.nih.gov/34385150/" style="text-decoration: none;">PubMed</a>

* **Next-generation sequencing diagnostics of bacteremia in pediatric sepsis.**  
  Schmoch T, Westhoff JH, Decker SO, Skarabis A, Hoffmann GF, **Dohna-Schwake C**, Felderhoff-Muser U, Skolik C, Feisst M, Klose C, Bruckner T, Luntz S, Weigand MA, Sohn K, Brenner T. *Medicine, 100(25):e26403*. <a href="https://pubmed.ncbi.nlm.nih.gov/34160425/" style="text-decoration: none;">PubMed</a>

* **Epidemiology, Diagnostics, and Management of Vein of Galen Malformation.**  
  Brevis Nunez F, **Dohna-Schwake C**. *Pediatric neurology, 119:50-55*. <a href="https://pubmed.ncbi.nlm.nih.gov/33895584/" style="text-decoration: none;">PubMed</a>

* **NAT2 polymorphisms as a cause of metamizole-induced agranulocytosis.**  
  Radulovic I, Dogan B, **Dohna-Schwake C**, Schundeln MM, Siffert W, Mohlendick B. *Pharmacogenetics and genomics, 31(6):140-143*. <a href="https://pubmed.ncbi.nlm.nih.gov/33675325/" style="text-decoration: none;">PubMed</a>

* **EVALI (E-cigarette or vaping product use associated lung injury): First case report of an adolescent in Europe.**  
  Schafer M, Steindor M, Stehling F, **Dohna-Schwake C**. *Pediatric pulmonology, 56(5):1274-1275*. <a href="https://pubmed.ncbi.nlm.nih.gov/33497507/" style="text-decoration: none;">PubMed</a>

* **Sedation for bronchoscopy in children: A prospective randomized double-blinded trial.**  
  Tschiedel E, Eckerland M, Felderhoff-Mueser U, **Dohna-Schwake C**, Stehling F. *Pediatric pulmonology, 56(5):1221-1229*. <a href="https://pubmed.ncbi.nlm.nih.gov/33336889/" style="text-decoration: none;">PubMed</a>

* **aEEG as a useful tool for neuromonitoring in critically ill children - Current evidence and knowledge gaps.**  
  **Bruns N**, Felderhoff-Muser U, **Dohna-Schwake C**. *Acta paediatrica (Oslo, Norway : 1992), 110(4):1132-1140*. <a href="https://pubmed.ncbi.nlm.nih.gov/33210762/" style="text-decoration: none;">PubMed</a>

* **Aminoglycosides were associated with higher rates of surgical patent ductus arteriosus closure in preterm infants.**  
  Marissen J, Erdmann H, Bockenholt K, Hoppenz M, Rausch TK, Hartel C, Herting E, Gopel W. *Acta paediatrica (Oslo, Norway : 1992), 110(3):826-832*. <a href="https://pubmed.ncbi.nlm.nih.gov/32810301/" style="text-decoration: none;">PubMed</a>


## 2020

* **Iatrogenic severe hyperglycemia due to parenteral administration of glucose in children - a case series.**  
  **Bruns N**, Grosse Lordemann A, Rasche T, Meyburg J, Kruger M, Wieg C, Gratopp A, Hoppenz M, Heitmann F, Hoppen T, Loffler G, Felderhoff-Muser U, **Dohna-Schwake C**. *Italian journal of pediatrics, 46(1):179*. <a href="https://pubmed.ncbi.nlm.nih.gov/33261643/" style="text-decoration: none;">PubMed</a>

* **Unusual Presentation of Polyautoimmunity and Renal Tubular Acidosis in an Adolescent With Hashimoto's Thyroiditis and Central Pontine Myelinolysis.**  
  **Bruns N**, Finkelberg I, Al-Attrach I, Hoyer PF, Buscher R. *Frontiers in endocrinology, 11:548877*. <a href="https://pubmed.ncbi.nlm.nih.gov/33162932/" style="text-decoration: none;">PubMed</a>

* **More Than 500 Kids Could Be Saved Each Year! Ten Consensus Actions to Improve Quality of Pediatric Resuscitation in DACH-Countries (Austria, Germany, and Switzerland).**  
  Jung P, Brenner S, Bachmann I, Both C, Cardona F, **Dohna-Schwake C**, Eich C, Eifinger F, Huth R, Heimberg E, Landsleitner B, Olivieri M, Sasse M, Weisner T, Wagner M, Warnke G, Ziegler B, Boettiger BW, Nadkarni V, Hoffmann F. *Frontiers in pediatrics, 8:549710*. <a href="https://pubmed.ncbi.nlm.nih.gov/33117762/" style="text-decoration: none;">PubMed</a>

* **Further evidence for POMK as candidate gene for WWS with meningoencephalocele.**  
  **Paul L**, Rupprich K, Della Marina A, Stein A, Elgizouli M, Kaiser FJ, Schweiger B, Koninger A, Iannaccone A, Hehr U, Kolbel H, Roos A, Schara-Schmidt U, Kuechler A. *Orphanet journal of rare diseases, 15(1):242*. <a href="https://pubmed.ncbi.nlm.nih.gov/32907597/" style="text-decoration: none;">PubMed</a>

* **How to Administer Near-Infrared Spectroscopy in Critically ill Neonates, Infants, and Children.**  
  **Bruns N**, Moosmann J, Munch F, **Dohna-Schwake C**, Woelfle J, Cesnjevar R, Dittrich S, Felderhoff-Muser U, Muller H. *Journal of visualized experiments : JoVE(162)*. <a href="https://pubmed.ncbi.nlm.nih.gov/32894267/" style="text-decoration: none;">PubMed</a>

* **Preterm birth and sustained inflammation: consequences for the neonate.**  
  Humberg A, Fortmann I, Siller B, Kopp MV, Herting E, Gopel W, Hartel C. *Seminars in immunopathology, 42(4):451-468*. <a href="https://pubmed.ncbi.nlm.nih.gov/32661735/" style="text-decoration: none;">PubMed</a>

* **Postnatal serum creatinine is elevated in preterm infants with PPROM-induced anhydramnios.**  
  **Bruns N**, Stahling AC, **Greve S**, Weiss C, Koninger A, Felderhoff-Muser U, Muller H. *Pediatrics and neonatology, 61(4):414-419*. <a href="https://pubmed.ncbi.nlm.nih.gov/32278744/" style="text-decoration: none;">PubMed</a>

* **Two-year outcome data suggest that less invasive surfactant administration (LISA) is safe. Results from the follow-up of the randomized controlled AMV (avoid mechanical ventilation) study.**  
  Herting E, Kribs A, Hartel C, von der Wense A, Weller U, Hoehn T, Vochem M, Moller J, Wieg C, Roth B, Gopel W. *European journal of pediatrics, 179(8):1309-1313*. <a href="https://pubmed.ncbi.nlm.nih.gov/32067100/" style="text-decoration: none;">PubMed</a>

* **Surgical necrotizing enterocolitis but not spontaneous intestinal perforation is associated with adverse neurological outcome at school age.**  
  Humberg A, Spiegler J, Fortmann MI, Zemlin M, Marissen J, Swoboda I, Rausch TK, Herting E, Gopel W, Hartel C. *Scientific reports, 10(1):2373*. <a href="https://pubmed.ncbi.nlm.nih.gov/32047169/" style="text-decoration: none;">PubMed</a>

* **aEEG Use in Pediatric Critical Care-An Online Survey.**  
  **Bruns N**, Felderhoff-Muser U, **Dohna-Schwake C**, Woelfle J, Muller H. *Frontiers in pediatrics, 8:3*. <a href="https://pubmed.ncbi.nlm.nih.gov/32039124/" style="text-decoration: none;">PubMed</a>

* **Correlation of lateral ventricular size and deep gray matter volume in MRI at term equivalent age with neurodevelopmental outcome at a corrected age of 24 months and with handedness in preterm infants.**  
  Storbeck T, **Bruns N**, Weiss C, Felderhoff-Muser U, Muller H. *European journal of pediatrics, 179(2):271-278*. <a href="https://pubmed.ncbi.nlm.nih.gov/31724086/" style="text-decoration: none;">PubMed</a>

* **Bacterial infections in children after liver transplantation: A single-center surveillance study of 345 consecutive transplantations.**  
  **Dohna Schwake C**, Guiddir T, Cuzon G, Benissa MR, Dubois C, Miatello J, Merchaoui Z, Durand P, Tissieres P. *Transplant infectious disease : an official journal of the Transplantation Society, 22(1):e13208*. <a href="https://pubmed.ncbi.nlm.nih.gov/31693773/" style="text-decoration: none;">PubMed</a>

* **Neurological outcome at 24 months corrected age of prematurely born infants after preterm premature rupture of membranes (PPROM) of at least 7 days: a two-center experience in Germany.**  
  Muller H, Storbeck T, Katzer D, **Bruns N**, Wossner-Stegmann G, Ai M, Koninger A, Muller A, Felderhoff-Muser U, Bagci S. *The journal of maternal-fetal & neonatal medicine : the official journal of the European Association of Perinatal Medicine, the Federation of Asia and Oceania Perinatal Societies, the International Society of Perinatal Obstetricians, 33(8):1315-1320*. <a href="https://pubmed.ncbi.nlm.nih.gov/30153748/" style="text-decoration: none;">PubMed</a>


## 2019

* **Research priorities in pediatric onco-critical care: an international Delphi consensus study.**  
  Soeteman M, Potratz J, Nielsen JSA, Willems J, Valla FV, Brierley J, Wosten-van Asperen RM. *Intensive care medicine, 45(11):1681-1683*. <a href="https://pubmed.ncbi.nlm.nih.gov/31444505/" style="text-decoration: none;">PubMed</a>

* **Amplitude-Integrated EEG for Neurological Assessment and Seizure Detection in a German Pediatric Intensive Care Unit.**  
  **Bruns N**, Sanchez-Albisua I, Weiss C, Tschiedel E, **Dohna-Schwake C**, Felderhoff-Muser U, Muller H. *Frontiers in pediatrics, 7:358*. <a href="https://pubmed.ncbi.nlm.nih.gov/31555625/" style="text-decoration: none;">PubMed</a>

* **PICU mortality of children with cancer admitted to pediatric intensive care unit a systematic review and meta-analysis.**  
  Wosten-van Asperen RM, van Gestel JPJ, van Grotel M, Tschiedel E, **Dohna-Schwake C**, Valla FV, Willems J, Angaard Nielsen JS, Krause MF, Potratz J, van den Heuvel-Eibrink MM, Brierley J. *Critical reviews in oncology/hematology, 142:153-163*. <a href="https://pubmed.ncbi.nlm.nih.gov/31404827/" style="text-decoration: none;">PubMed</a>

* **A DELPHI study on aspects of study design to overcome knowledge gaps on the burden of disease caused by serogroup B invasive meningococcal disease.**  
  Marten O, Koerber F, Bloom D, Bullinger M, Buysse C, Christensen H, De Wals P, **Dohna-Schwake C**, Henneke P, Kirchner M, Knuf M, Lawrenz B, Monteiro AL, Sevilla JP, Van de Velde N, Welte R, Wright C, Greiner W. *Health and quality of life outcomes, 17(1):87*. <a href="https://pubmed.ncbi.nlm.nih.gov/31118091/" style="text-decoration: none;">PubMed</a>

* **Visual Distraction During Sleep Induction with Propofol in Pediatric Patients - Can Cartoon Watching Influence the Sedative Dose?**  
  Tschiedel E, Beckers K, Grosse-Lordemann A, Basu O, Felderhoff-Muser U, **Dohna-Schwake C**. *Klinische Padiatrie, 231(1):35-40*. <a href="https://pubmed.ncbi.nlm.nih.gov/30231262/" style="text-decoration: none;">PubMed</a>


## 2018

* **Relationship between brain function (aEEG) and brain structure (MRI) and their predictive value for neurodevelopmental outcome of preterm infants.**  
  Huning B, Storbeck T, **Bruns N**, Dransfeld F, Hobrecht J, Karpienski J, Sirin S, Schweiger B, Weiss C, Felderhoff-Muser U, Muller H. *European journal of pediatrics, 177(8):1181-1189*. <a href="https://pubmed.ncbi.nlm.nih.gov/29789947/" style="text-decoration: none;">PubMed</a>

* **Antifungal Treatment and Outcome in Very Low Birth Weight Infants: A Population-based Observational Study of the German Neonatal Network.**  
  Fortmann I, Hartz A, Paul P, Pulzer F, Muller A, Bottger R, Proquitte H, Dawczynski K, Simon A, Rupp J, Herting E, Gopel W, Hartel C. *The Pediatric infectious disease journal, 37(11):1165-1171*. <a href="https://pubmed.ncbi.nlm.nih.gov/29601449/" style="text-decoration: none;">PubMed</a>


## 2017

* **Health of VLBW infants in Germany at five years of age: What do parents describe?**  
  Spiegler J, Stichtenoth G, Konig IR, Herting E, Gopel W. *Early human development, 115:88-92*. <a href="https://pubmed.ncbi.nlm.nih.gov/29024833/" style="text-decoration: none;">PubMed</a>

* **Application of an Amplitude-integrated EEG Monitor (Cerebral Function Monitor) to Neonates.**  
  **Bruns N**, Blumenthal S, Meyer I, Klose-Verschuur S, Felderhoff-Muser U, Muller H. *Journal of visualized experiments : JoVE(127)*. <a href="https://pubmed.ncbi.nlm.nih.gov/28930990/" style="text-decoration: none;">PubMed</a>

* **The association of mannose-binding lectin 2 polymorphisms with outcome in very low birth weight infants.**  
  Hartz A, Pagel J, Humberg A, Preuss M, Schreiter L, Rupp J, Figge J, Karsten CM, Nurnberg P, Herting E, Gopel W, Hartel C. *PloS one, 12(5):e0178032*. <a href="https://pubmed.ncbi.nlm.nih.gov/28558032/" style="text-decoration: none;">PubMed</a>

* **Delivery mode and intraventricular hemorrhage risk in very-low-birth-weight infants: Observational data of the German Neonatal Network.**  
  Humberg A, Hartel C, Paul P, Hanke K, Bossung V, Hartz A, Fasel L, Rausch TK, Rody A, Herting E, Gopel W. *European journal of obstetrics, gynecology, and reproductive biology, 212:144-149*. <a href="https://pubmed.ncbi.nlm.nih.gov/28363188/" style="text-decoration: none;">PubMed</a>

* **Comparison of two common aEEG classifications for the prediction of neurodevelopmental outcome in preterm infants.**  
  **Bruns N**, Dransfeld F, Huning B, Hobrecht J, Storbeck T, Weiss C, Felderhoff-Muser U, Muller H. *European journal of pediatrics, 176(2):163-171*. <a href="https://pubmed.ncbi.nlm.nih.gov/27924356/" style="text-decoration: none;">PubMed</a>

* **Glucocorticoid Receptor Gene Variants and Neonatal Outcome in Very-Low-Birth-Weight Preterm Infants.**  
  Schreiner C, Schreiner F, Hartel C, Heckmann M, Heep A, Bartmann P, Woelfle J, Muller A, Herting E, Gopel W. *Neonatology, 111(1):22-29*. <a href="https://pubmed.ncbi.nlm.nih.gov/27509264/" style="text-decoration: none;">PubMed</a>


## 2016

* **Clinical Evaluation of the First Automated Assay for the Detection of Stimulating TSH Receptor Autoantibodies.**  
  Allelein S, Ehlers M, **Goretzki S**, Hermsen D, Feldkamp J, Haase M, Dringenberg T, Schmid C, Hautzel H, Schott M. *Hormone and metabolic research = Hormon- und Stoffwechselforschung = Hormones et metabolisme, 48(12):795-801*. <a href="https://pubmed.ncbi.nlm.nih.gov/27923250/" style="text-decoration: none;">PubMed</a>

* **Media Stories on NICU Outbreaks Lead to an Increased Prescription Rate of Third-Line Antibiotics in the Community of Neonatal Care.**  
  Hartel C, Hartz A, Bahr L, Gille C, Gortner L, Simon A, Orlikowsky T, Muller A, Korner T, Henneke P, Haase R, Zemlin M, Viemann D, Gebauer C, Thome U, Ziegler A, Rupp J, Herting E, Gopel W. *Infection control and hospital epidemiology, 37(8):924-930*. <a href="https://pubmed.ncbi.nlm.nih.gov/27143176/" style="text-decoration: none;">PubMed</a>

* **NOD2 Loss-of-Function Mutations and Risks of Necrotizing Enterocolitis or Focal Intestinal Perforation in Very Low-birth-weight Infants.**  
  Hartel C, Hartz A, Pagel J, Rupp J, Stein A, Kribs A, Muller A, Haase R, Gille C, Bottger R, Kittel J, Jensen R, Wieg C, Herting E, Gopel W. *Inflammatory bowel diseases, 22(2):249-56*. <a href="https://pubmed.ncbi.nlm.nih.gov/26752461/" style="text-decoration: none;">PubMed</a>

* **Does Breastmilk Influence the Development of Bronchopulmonary Dysplasia?**  
  Spiegler J, Preuss M, Gebauer C, Bendiks M, Herting E, Gopel W. *The Journal of pediatrics, 169:76-80.e4*. <a href="https://pubmed.ncbi.nlm.nih.gov/26621048/" style="text-decoration: none;">PubMed</a>


## 2015

* **Short-term outcome of very-low-birthweight infants with arterial hypotension in the first 24 h of life.**  
  Faust K, Hartel C, Preuss M, Rabe H, Roll C, Emeis M, Wieg C, Szabo M, Herting E, Gopel W. *Archives of disease in childhood. Fetal and neonatal edition, 100(5):F388-92*. <a href="https://pubmed.ncbi.nlm.nih.gov/26199082/" style="text-decoration: none;">PubMed</a>

* **Electrocortical Activity at 7 Days of Life is Affected in Extremely Premature Infants with Patent Ductus Arteriosus.**  
  **Bruns N**, Metze B, Buhrer C, Felderhoff-Muser U, Huseman D. *Klinische Padiatrie, 227(5):264-8*. <a href="https://pubmed.ncbi.nlm.nih.gov/25875399/" style="text-decoration: none;">PubMed</a>

* **Length and weight of very low birth weight infants in Germany at 2 years of age: does it matter at what age they start complementary food?**  
  Spiegler J, Eisemann N, Ehlers S, Orlikowsky T, Kannt O, Herting E, Gopel W. *European journal of clinical nutrition, 69(6):662-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/25872910/" style="text-decoration: none;">PubMed</a>

* **Preterm prelabor rupture of membranes and outcome of very-low-birth-weight infants in the German Neonatal Network.**  
  Hanke K, Hartz A, Manz M, Bendiks M, Heitmann F, Orlikowsky T, Muller A, Olbertz D, Kuhn T, Siegel J, von der Wense A, Wieg C, Kribs A, Stein A, Pagel J, Herting E, Gopel W, Hartel C. *PloS one, 10(4):e0122564*. <a href="https://pubmed.ncbi.nlm.nih.gov/25856083/" style="text-decoration: none;">PubMed</a>

* **Less invasive surfactant administration is associated with improved pulmonary outcomes in spontaneously breathing preterm infants.**  
  Gopel W, Kribs A, Hartel C, Avenarius S, Teig N, Groneck P, Olbertz D, Roll C, Vochem M, Weller U, von der Wense A, Wieg C, Wintgens J, Preuss M, Ziegler A, Roth B, Herting E. *Acta paediatrica (Oslo, Norway : 1992), 104(3):241-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/25474712/" style="text-decoration: none;">PubMed</a>


## 2014

* **Impairment of bone health in pediatric patients with hemolytic anemia.**  
  Schundeln MM, **Goretzki SC**, Hauffa PK, Wieland R, Bauer J, Baeder L, Eggert A, Hauffa BP, Grasemann C. *PloS one, 9(10):e108400*. <a href="https://pubmed.ncbi.nlm.nih.gov/25299063/" style="text-decoration: none;">PubMed</a>

* **Mitochondrial mutation m.1555A>G as a risk factor for failed newborn hearing screening in a large cohort of preterm infants.**  
  Gopel W, Berkowski S, Preuss M, Ziegler A, Kuster H, Felderhoff-Muser U, Gortner L, Mogel M, Hartel C, Herting E. *BMC pediatrics, 14:210*. <a href="https://pubmed.ncbi.nlm.nih.gov/25155176/" style="text-decoration: none;">PubMed</a>

* **Prophylactic use of Lactobacillus acidophilus/Bifidobacterium infantis probiotics and outcome in very low birth weight infants.**  
  Hartel C, Pagel J, Rupp J, Bendiks M, Guthmann F, Rieger-Fackeldey E, Heckmann M, Franz A, Schiffmann JH, Zimmermann B, Hepping N, von der Wense A, Wieg C, Herting E, Gopel W. *The Journal of pediatrics, 165(2):285-289.e1*. <a href="https://pubmed.ncbi.nlm.nih.gov/24880888/" style="text-decoration: none;">PubMed</a>

* **Risk for late-onset blood-culture proven sepsis in very-low-birth weight infants born small for gestational age: a large multicenter study from the German Neonatal Network.**  
  Troger B, Gopel W, Faust K, Muller T, Jorch G, Felderhoff-Muser U, Gortner L, Heitmann F, Hoehn T, Kribs A, Laux R, Roll C, Emeis M, Mogel M, Siegel J, Vochem M, von der Wense A, Wieg C, Herting E, Hartel C. *The Pediatric infectious disease journal, 33(3):238-43*. <a href="https://pubmed.ncbi.nlm.nih.gov/24030351/" style="text-decoration: none;">PubMed</a>


## 2013

* **Use of analgesic and sedative drugs in VLBW infants in German NICUs from 2003-2010.**  
  Mehler K, Oberthuer A, Haertel C, Herting E, Roth B, Goepel W. *European journal of pediatrics, 172(12):1633-9*. <a href="https://pubmed.ncbi.nlm.nih.gov/23877637/" style="text-decoration: none;">PubMed</a>

* **Polymorphisms in FTO and MAF Genes and Birth Weight, BMI, Ponderal Index, Weight Gain in a Large Cohort of Infants with a Birth Weight below 1500 Grams.**  
  Haller S, Spiegler J, Hemmelmann C, Kuster H, Vochem M, Moller J, Muller D, Kribs A, Hoehn T, Hartel C, Herting E, Gopel W. *PloS one, 8(6):e66331*. <a href="https://pubmed.ncbi.nlm.nih.gov/23840443/" style="text-decoration: none;">PubMed</a>

* **Pregnancy risk factors for very premature delivery: what role do hypertension, obesity and diabetes play?**  
  Spiegler J, Stichtenoth G, Weichert J, Konig IR, Schlaud M, V D Wense A, Olbertz D, Gurth H, Schiffmann JH, Bohnhorst B, Gortner L, Herting E, Gopel W. *Archives of gynecology and obstetrics, 288(1):57-64*. <a href="https://pubmed.ncbi.nlm.nih.gov/23400353/" style="text-decoration: none;">PubMed</a>

* **Very low birth weight infants after discharge: What do parents describe?**  
  Spiegler J, Schlaud M, Konig IR, Teig N, Hubert M, Herting E, Gopel W. *Early human development, 89(5):343-7*. <a href="https://pubmed.ncbi.nlm.nih.gov/23266151/" style="text-decoration: none;">PubMed</a>


## 2012

* **Causes of delivery and outcomes of very preterm twins stratified to zygosity.**  
  Spiegler J, Hartel C, Schulz L, von Wurmb-Schwark N, Hoehn T, Kribs A, Kuster H, Siegel J, Wieg C, Weichert J, Herting E, Gopel W. *Twin research and human genetics : the official journal of the International Society for Twin Studies, 15(4):532-6*. <a href="https://pubmed.ncbi.nlm.nih.gov/22853920/" style="text-decoration: none;">PubMed</a>

* **Epidemic microclusters of blood-culture proven sepsis in very-low-birth weight infants: experience of the German Neonatal Network.**  
  Hartel C, Faust K, Avenarius S, Bohnhorst B, Emeis M, Gebauer C, Groneck P, Heitmann F, Hoehn T, Hubert M, Kribs A, Kuster H, Laux R, Mogel M, Muller D, Olbertz D, Roll C, Siegel J, Stein A, Vochem M, Weller U, von der Wense A, Wieg C, Wintgens J, Hemmelmann C, Simon A, Herting E, Gopel W. *PloS one, 7(6):e38304*. <a href="https://pubmed.ncbi.nlm.nih.gov/22768043/" style="text-decoration: none;">PubMed</a>

* **ATP-binding cassette member A3 (E292V) gene mutation and pulmonary morbidity in very-low-birth-weight infants.**  
  Hartel C, Felderhoff-Muser U, Gebauer C, Hoehn T, Kribs A, Laux R, Moller J, Segerer H, Teig N, von der Wense A, Wieg C, Stichtenoth G, Herting E, Gopel W. *Acta paediatrica (Oslo, Norway : 1992), 101(4):380-3*. <a href="https://pubmed.ncbi.nlm.nih.gov/22145626/" style="text-decoration: none;">PubMed</a>


## 2011

* **Avoidance of mechanical ventilation by surfactant treatment of spontaneously breathing preterm infants (AMV): an open-label, randomised, controlled trial.**  
  Gopel W, Kribs A, Ziegler A, Laux R, Hoehn T, Wieg C, Siegel J, Avenarius S, von der Wense A, Vochem M, Groneck P, Weller U, Moller J, Hartel C, Haller S, Roth B, Herting E. *Lancet (London, England), 378(9803):1627-34*. <a href="https://pubmed.ncbi.nlm.nih.gov/21963186/" style="text-decoration: none;">PubMed</a>

* **Tumor necrosis factor-alpha promoter -308 G/A polymorphism and susceptibility to sepsis in very-low-birth-weight infants.**  
  Hartel C, Hemmelmann C, Faust K, Gebauer C, Hoehn T, Kribs A, Laux R, Nikischin W, Segerer H, Teig N, von der Wense A, Wieg C, Herting E, Gopel W. *Critical care medicine, 39(5):1190-5*. <a href="https://pubmed.ncbi.nlm.nih.gov/21317641/" style="text-decoration: none;">PubMed</a>




defaultdict(list,
            {'2023': [{'PMID': '36670127',
               'OWN': 'NLM',
               'STAT': 'MEDLINE',
               'DCOM': '20230124',
               'LR': '20230316',
               'IS': '2045-2322 (Electronic) 2045-2322 (Linking)',
               'VI': '13',
               'IP': '1',
               'DP': '2023 Jan 20',
               'TI': 'A comparison of pediatric inflammatory multisystem syndrome temporarily-associated with SARS-CoV-2 and Kawasaki disease.',
               'PG': '1173',
               'LID': '10.1038/s41598-022-26832-5 [doi] 1173',
               'AB': 'The connection between Pediatric Inflammatory Multisystem Syndrome (PIMS) and Kawasaki Disease (KD) is not yet fully understood. Using the same national registry, clinical features and outcome of children hospitalized in Germany, and Innsbruck (Austria) were compared. Reported to the registry were 395 PIMS and 69 KD hospitalized patients. Patient age in PIMS cases was higher than in KD case